In [1]:
import random
import time
import urllib.request
from bs4 import BeautifulSoup as bs
import re
import jieba
import pandas as pd
import numpy as np
import requests
import os

In [34]:
#reset dataframe
#新闻标题，日期，发布人，撰稿人，新闻类别，新闻内容
# news_detail_df = pd.DataFrame(columns=['news title', 'news publisher', 'news writer',
#                                         'news content', 'news date', 'news category'])
# news_detail_df

,news title,news publisher,news writer,news content,news date,news category


In [3]:
# read existed csv file
news_detail_df = pd.read_csv('result_detail.csv')
news_detail_df_link = news_detail_df['link'].values.tolist()

In [4]:
news_detail_df

,title,publisher,writer,content,date,category,link
0,苏州市分公司开展2020年度安全生产培训,发布人: 徐可,撰稿人:,7月4日，为进一步落实企业安全生产主体责任，提升企业安全生产能力和水平，苏州市分公司在滨河...,"周一, 07/06/2020 - 14:40",新闻类别:,/node/84
1,苏州联通干字当头，全力保障沪苏通长江公铁大桥正式通车,发布人: 徐可,撰稿人:,7月1日上午11时45分，随着沪苏通铁路首趟列车缓缓使入张家港站，苏州常熟、太仓、张家港三...,"周一, 07/06/2020 - 22:15",新闻类别:,/node/85
2,明方向 知不足 打胜仗——苏州市分公司召开7月工作部署会,发布人: 徐可,撰稿人:,7月6日，苏州市分公司召开7月工作部署会，总结6月得失，明确7月方向。公司领导班子、各区县...,"周二, 07/07/2020 - 15:56",新闻类别:,/node/87
3,“全力以赴，奋战三季度”——市区分公司快速落实市分公司周例会工作要求,发布人: 徐可,撰稿人:,7月13日晚，市区分公司在滨河路1300号309会议室召集各条线部门、网格、片区经理，快速...,"周二, 07/14/2020 - 12:19",新闻类别:,/node/103
4,张家港分公司召开“7月第二周周例会”,发布人: 陈莹,撰稿人:,7月13日晚，张家港分公司召开“7月第二周周例会”，各部门、网格（中心）对上周的工作情况进...,"周二, 07/14/2020 - 15:22",新闻类别:,/node/106
...,...,...,...,...,...,...,...
2155,使命担当、数字防疫 ——太仓政企青年突击队在行动,发布人: 王晓晨,撰稿人: 王晓晨,为了充分发挥抗“疫”先锋作用，以最快的速度响应落实政府防疫需求，确保疫情防疫工作有效...,"周一, 02/21/2022 - 17:25",新闻类别: 党建,/node/4147
2156,双胜利 站出来 冲在前—— 公众条线稳发展保产保收,发布人: 陆瑶雯,撰稿人: 陆瑶雯,面对疫情严峻形势，为保发展产能不掉队，渠道运营中心在公司党委带领下，快速响应、调整战略...,"周一, 02/21/2022 - 17:48",新闻类别: 会议活动（内部）,/node/4148
2157,“疫情就是命令，防控就是责任”苏州联通职能线党员青年突击队在行动,发布人: 吴荣昌,撰稿人: 吴荣昌,2月13日以来，苏州市突发新冠肺炎疫情，防控形势严峻复杂。为迅速落实市委市政府对疫情防控工作...,"周一, 02/21/2022 - 17:51",新闻类别: 党建,/node/4149
2158,张家港分公司召开第三支队伍 “外呼突击队”工作部署会,发布人: 陈莹,撰稿人: 陆惠,2月21日晚，张家港公司召开“外呼突击队”工作部署会，快速落实市分工作要求，部署疫情下价值...,"周二, 02/22/2022 - 09:08",新闻类别: 会议活动（内部）,/node/4151


In [15]:
news_detail_df_link

['/node/84',
 '/node/85',
 '/node/87',
 '/node/103',
 '/node/106',
 '/node/115',
 '/node/116',
 '/node/117',
 '/node/118',
 '/node/121',
 '/node/122',
 '/node/138',
 '/node/140',
 '/node/143',
 '/node/147',
 '/node/149',
 '/node/152',
 '/node/153',
 '/node/156',
 '/node/159',
 '/node/160',
 '/node/165',
 '/node/167',
 '/node/178',
 '/node/175',
 '/node/176',
 '/node/177',
 '/node/179',
 '/node/181',
 '/node/184',
 '/node/185',
 '/node/186',
 '/node/187',
 '/node/189',
 '/node/190',
 '/node/191',
 '/node/192',
 '/node/193',
 '/node/194',
 '/node/195',
 '/node/196',
 '/node/199',
 '/node/200',
 '/node/203',
 '/node/206',
 '/node/209',
 '/node/210',
 '/node/211',
 '/node/214',
 '/node/215',
 '/node/216',
 '/node/226',
 '/node/227',
 '/node/228',
 '/node/230',
 '/node/236',
 '/node/238',
 '/node/239',
 '/node/240',
 '/node/241',
 '/node/246',
 '/node/248',
 '/node/249',
 '/node/250',
 '/node/251',
 '/node/252',
 '/node/254',
 '/node/260',
 '/node/266',
 '/node/271',
 '/node/272',
 '/node/2

In [16]:
#read existed news link
df_news_ = pd.read_csv('result.csv')
links_exist = df_news_['news link'].values.tolist()

In [17]:
links_exist

['/node/3866',
 '/node/3865',
 '/node/3863',
 '/node/3860',
 '/node/3861',
 '/node/3859',
 '/node/3858',
 '/node/3857',
 '/node/3856',
 '/node/3854',
 '/node/3852',
 '/node/3851',
 '/node/3849',
 '/node/3847',
 '/node/3846',
 '/node/3845',
 '/node/3844',
 '/node/3842',
 '/node/3841',
 '/node/3840',
 '/node/3839',
 '/node/3838',
 '/node/3837',
 '/node/3835',
 '/node/3834',
 '/node/3830',
 '/node/3829',
 '/node/3827',
 '/node/3826',
 '/node/3825',
 '/node/3824',
 '/node/3823',
 '/node/3821',
 '/node/3818',
 '/node/3816',
 '/node/3815',
 '/node/3814',
 '/node/3812',
 '/node/3809',
 '/node/3808',
 '/node/3806',
 '/node/3805',
 '/node/3803',
 '/node/3802',
 '/node/3799',
 '/node/3798',
 '/node/3797',
 '/node/3796',
 '/node/3794',
 '/node/3792',
 '/node/3791',
 '/node/3790',
 '/node/3789',
 '/node/3787',
 '/node/3785',
 '/node/3783',
 '/node/3782',
 '/node/3781',
 '/node/3780',
 '/node/3778',
 '/node/3776',
 '/node/3775',
 '/node/3773',
 '/node/3772',
 '/node/3771',
 '/node/3768',
 '/node/37

In [7]:
#function to return html data according to url link
def return_html(num):
    url = "http://10.201.112.8"+num
    head = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:93.0) Gecko/20100101 Firefox/93.0'}
    request = urllib.request.Request(url=url, headers=head)
    resp = urllib.request.urlopen(request)
    html_data = resp.read().decode('utf-8')
    return html_data

In [8]:
#function to parse html to dataframe
def parse_html(html, node_num):
    news_title = []
    news_publisher = []
    news_writer = []
    news_contents = []
    news_date = []
    news_category = []
    soup = bs(html, 'html.parser')
    news_title_ = soup.find_all('a', rel = 'bookmark')
    news_title.append(news_title_[0].text.strip())
    news_info_ = soup.find_all('div', class_ = 'article-create-date')
    news_date_ = news_info_[0]
    news_date_ = news_date_.find_all('span')[1]
    for i in news_date_:
        news_date.append(i.text)
    news_publisher_ = news_info_[1].text.strip()
    news_publisher_ = re.sub('\xa0','', news_publisher_)
    news_publisher.append(news_publisher_)
    news_writer_ = news_info_[2].text.strip()
    news_writer_ = re.sub('\xa0','',news_writer_)
    news_writer.append(news_writer_)
    news_category_ = news_info_[3].text.strip()
    news_category_ = re.sub('\xa0','', news_category_)
    news_category.append(news_category_)
    news_content_ = soup.find_all('p')
    for i in news_content_:
        string_ = i.text
        string_ = re.sub('\xa0','',string_)
        news_contents.append(string_)
        news_content = ''.join(news_contents)
    news_detail_dict = {'news title':news_title, 'news date': news_date, 'news publisher': news_publisher,
                        'news writer': news_writer, 'news category': news_category, 'news content':news_content,
                        'news link': node_num}
    news_df = pd.DataFrame(news_detail_dict)
    return news_df
    # return news_detail_dict

In [9]:
#function to download web images
def get_img(html, node_num):
    soup = bs(html, 'html.parser')
    all_links = soup.find_all('img')
    for i in range(2,len(all_links)):
        url = 'http://10.201.112.8'+all_links[i]['src']
        name = str(i-1)
        path = "D:/download_image"+node_num+"/"
        if not os.path.exists(path):
            os.makedirs(path)
        urllib.request.urlretrieve(url,path+name+".jpg")
        print("-----downloading-----"+node_num)
    print("----------done--------------")

In [18]:
#main program
for i in range(20, -1, -1):
    html_data = return_html(links_exist[i])
    df = parse_html(html_data, links_exist[i])
    news_detail_df_new = df[~df['news link'].isin(news_detail_df_link)]
    news_detail_df = pd.concat([news_detail_df, news_detail_df_new], axis=0)
    if links_exist[i] not in news_detail_df_link:
        get_img(html_data, links_exist[i])
    sleeptime = random.randint(8,16)
    time.sleep(sleeptime)

-----downloading-----/node/3861
-----downloading-----/node/3861
----------done--------------
-----downloading-----/node/3860
----------done--------------
----------done--------------
-----downloading-----/node/3865
----------done--------------
-----downloading-----/node/3866
----------done--------------


In [19]:
news_detail_df

,news title,news publisher,news writer,news content,news date,news category,news link
0,苏州市分公司开展2020年度安全生产培训,发布人: 徐可,撰稿人:,7月4日，为进一步落实企业安全生产主体责任，提升企业安全生产能力和水平，苏州市分公司在滨河...,"周一, 07/06/2020 - 14:40",新闻类别:,/node/84
1,苏州联通干字当头，全力保障沪苏通长江公铁大桥正式通车,发布人: 徐可,撰稿人:,7月1日上午11时45分，随着沪苏通铁路首趟列车缓缓使入张家港站，苏州常熟、太仓、张家港三...,"周一, 07/06/2020 - 22:15",新闻类别:,/node/85
2,明方向 知不足 打胜仗——苏州市分公司召开7月工作部署会,发布人: 徐可,撰稿人:,7月6日，苏州市分公司召开7月工作部署会，总结6月得失，明确7月方向。公司领导班子、各区县...,"周二, 07/07/2020 - 15:56",新闻类别:,/node/87
3,“全力以赴，奋战三季度”——市区分公司快速落实市分公司周例会工作要求,发布人: 徐可,撰稿人:,7月13日晚，市区分公司在滨河路1300号309会议室召集各条线部门、网格、片区经理，快速...,"周二, 07/14/2020 - 12:19",新闻类别:,/node/103
4,张家港分公司召开“7月第二周周例会”,发布人: 陈莹,撰稿人:,7月13日晚，张家港分公司召开“7月第二周周例会”，各部门、网格（中心）对上周的工作情况进...,"周二, 07/14/2020 - 15:22",新闻类别:,/node/106
...,...,...,...,...,...,...,...
0,工业园区分公司开展2022年春节引流交付跟单演练,发布人: 吴静文,撰稿人: 曾诚昊,2022年春节引流交付战役即将打响，为全力保障春节引流交付的高质转化，全面提升员工兼...,"周一, 01/17/2022 - 09:25",新闻类别: 会议活动（内部）,/node/3861
0,昆山分公司政企线召开苏康码防疫产品攻坚调度会,发布人: 陆群芳,撰稿人: 陆兵,1月15日上午，为加快昆山苏康码防疫产品规模突破，昆山分公司政企线组织召开苏康码防疫产品攻...,"周一, 01/17/2022 - 09:51",新闻类别: 会议活动（内部）,/node/3860
0,客户价值运营中心组织开展群组化智家包及移移融合专项推进会,发布人: 余珍璐,撰稿人: 余珍璐,为强化一线人员对群组化智家包和移移融合营销产品的熟知度，加快推进落实。1月14日下午，...,"周一, 01/17/2022 - 10:17",新闻类别: 会议活动（内部）,/node/3863
0,苏州联通党群工作部宣贯省公司工作会议精神,发布人: 廖姮,撰稿人: 廖姮,1月17日，苏州联通党群工作部召开会议，贯彻落实集团公司“1+8+2”战略规划体系、省公司...,"周一, 01/17/2022 - 12:23",新闻类别: 会议活动（内部）,/node/3865


In [20]:
news_detail_df.to_csv('result_detail.csv', encoding='utf_8_sig', index=0)

In [27]:
for i in range(len(links_exist), len(links_exist)-100, -1):
    print(i)





1730
1729
1728
1727
1726
1725
1724
1723
1722
1721
1720
1719
1718
1717
1716
1715
1714
1713
1712
1711
1710
1709
1708
1707
1706
1705
1704
1703
1702
1701
1700
1699
1698
1697
1696
1695
1694
1693
1692
1691
1690
1689
1688
1687
1686
1685
1684
1683
1682
1681
1680
1679
1678
1677
1676
1675
1674
1673
1672
1671
1670
1669
1668
1667
1666
1665
1664
1663
1662
1661
1660
1659
1658
1657
1656
1655
1654
1653
1652
1651
1650
1649
1648
1647
1646
1645
1644
1643
1642
1641
1640
1639
1638
1637
1636
1635
1634
1633
1632
1631


In [30]:
links_exist[1730]
# for i in range(2,len(all_links)):
#     url = 'http://10.201.112.8'+all_links[i]['src']
#     name = str(i-1)
#     path = "D://download_image/"
#     urllib.request.urlretrieve(url,path+name+".jpg")
#     print("-----------downloading------------")
# print("----------done--------------")

IndexError: list index out of range

In [115]:
news_detail_df





,news title,news date,news publisher,news writer,news category,news content
0,明方向 知不足 打胜仗——苏州市分公司召开7月工作部署会,"周二, 07/07/2020 - 15:56",发布人: 徐可,撰稿人:,新闻类别:,7月6日，苏州市分公司召开7月工作部署会，总结6月得失，明确7月方向。公司领导班子、各区县...


In [ ]:
# soup = bs(html_data, 'html.parser')
# news_info_ = soup.find_all('div', class_ = 'article-create-date')
#

In [ ]:
# news_date = news_info_[0]
# news_date = news_date.find_all('span')[1]
# for i in news_date:
#     print(i.text)
#

In [ ]:
# news_publisher_ = news_info_[1]
# news_publisher_ = news_publisher_.text.strip()
# news_publisher_
# news_publisher_ = re.sub('\xa0','', news_publisher_)
# news_publisher_
#

In [ ]:
# news_writer_ = news_info_[2].text.strip()
# news_writer_ = re.sub('\xa0','',news_writer_)
# news_writer_
#

In [ ]:
# news_category_ = news_info_[3].text.strip()
# news_category_ = re.sub('\xa0','', news_category_)
# news_category_
#

In [ ]:
# news_content_ = soup.find_all('p')
# for i in news_content_:
#     print(i.text)